<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-Race-Selection" data-toc-modified-id="Get-Race-Selection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get Race Selection</a></span><ul class="toc-item"><li><span><a href="#regattas" data-toc-modified-id="regattas-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>regattas</a></span></li><li><span><a href="#races" data-toc-modified-id="races-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>races</a></span></li></ul></li><li><span><a href="#Fill-columns-with-extra-information-to-complete-database" data-toc-modified-id="Fill-columns-with-extra-information-to-complete-database-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fill columns with extra information to complete database</a></span><ul class="toc-item"><li><span><a href="#Update-final-rank-in-race_comp" data-toc-modified-id="Update-final-rank-in-race_comp-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Update final rank in race_comp</a></span></li><li><span><a href="#Get-chosen-side-of-competitors-for-leg-1" data-toc-modified-id="Get-chosen-side-of-competitors-for-leg-1-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get chosen side of competitors for leg 1</a></span></li><li><span><a href="#Determining-Course-Area" data-toc-modified-id="Determining-Course-Area-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Determining Course Area</a></span></li><li><span><a href="#Number-of-competitors-per-race-and-their-relative-rank" data-toc-modified-id="Number-of-competitors-per-race-and-their-relative-rank-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Number of competitors per race and their relative rank</a></span></li><li><span><a href="#Categorize-Sailing-Style-per-Race" data-toc-modified-id="Categorize-Sailing-Style-per-Race-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Categorize Sailing Style per Race</a></span></li><li><span><a href="#Positions-Start-Line" data-toc-modified-id="Positions-Start-Line-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Positions Start Line</a></span></li><li><span><a href="#Relative-Speeds" data-toc-modified-id="Relative-Speeds-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Relative Speeds</a></span></li></ul></li><li><span><a href="#Relative-traveled-distance" data-toc-modified-id="Relative-traveled-distance-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Relative traveled distance</a></span><ul class="toc-item"><li><span><a href="#positions_view" data-toc-modified-id="positions_view-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>positions_view</a></span></li><li><span><a href="#Calculate-Correlations" data-toc-modified-id="Calculate-Correlations-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Calculate Correlations</a></span></li></ul></li><li><span><a href="#Extra" data-toc-modified-id="Extra-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extra</a></span><ul class="toc-item"><li><span><a href="#Usefull-other-queries" data-toc-modified-id="Usefull-other-queries-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Usefull other queries</a></span></li><li><span><a href="#get-all-table-and-column-names-in-db" data-toc-modified-id="get-all-table-and-column-names-in-db-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>get all table and column names in db</a></span></li></ul></li></ul></div>

# Get Race Selection

In [26]:
###############################
## Tabellen vullen
###############################
import os, sys, inspect, json
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from lib.Utilities.globalVar import *
import lib.TransformData.combineData as combd
cd = combd.combineData()
import Download.download as download
from Utilities.converter import *

import pandas as pd
import numpy as np
from math import *
import urllib, json, requests
import pyodbc
from sqlalchemy import *
from sqlalchemy.exc import IntegrityError, InternalError
from sqlalchemy.types import BINARY, VARCHAR
from sqlalchemy import event
from sqlalchemy.types import TypeDecorator, CHAR
from sqlalchemy.dialects.postgresql import UUID
import uuid

In [27]:

server = 'sic-match-analysis.database.windows.net'
database = 'match-analysis'
username = 'sic-admin'
password = 'ZeilenIsLeuk!'
driver= '{ODBC Driver 17 for SQL Server}'
connection_str = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password


cnxn = pyodbc.connect(connection_str)
cursor = cnxn.cursor()

quoted = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))


@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True
        cursor.commit()

In [28]:
event  = [
    {'name' :  'hwcs2020-round1', 'AllRegattas': True, 'regattaNameContaining': 'HWCS 2020 Round 1', 'raceNameContaining': ''}
         ,{'name' : 'www', 'AllRegattas' : False, 'regattaNameContaining': 'Enoshima', 'raceNameContaining': ''}
         ,
    {'name' :  'tokyo2019', 'AllRegattas': True, 'regattaNameContaining': 'Tokyo', 'raceNameContaining': ''}
]
events = [{'name' :  'www', 'regattaNameContaining': 'WCS 2019 Genoa - 49er', 'raceNameContaining': ''},
         {'name' : 'www', 'regattaNameContaining': 'WCS 2019 Genoa - Nacra', 'raceNameContaining' : 
          ''}]
eventSelection = event

addNewRaces = False

## regattas

In [17]:
## TABLE 1 regattas
sql = """
SELECT * from regattas
"""
df = pd.read_sql(sql, con = engine)

reg = df[:0]

for serv in eventSelection: 
    temp = df[df['regatta'].str.contains(serv['regattaNameContaining'])]
    reg = reg.append(temp)
    

reg = reg.reset_index(drop = True)


In [16]:
reg

,regatta,boatclass,courseAreaId
0,HWCS 2020 Round 1 - 470 Men,470,180a9561-4034-4cc6-aaa6-698338bfd5cc
1,HWCS 2020 Round 1 - 470 Women,470,180a9561-4034-4cc6-aaa6-698338bfd5cc
2,HWCS 2020 Round 1 - 49er,49er,180a9561-4034-4cc6-aaa6-698338bfd5cc
3,HWCS 2020 Round 1 - 49erFX,49erFX,180a9561-4034-4cc6-aaa6-698338bfd5cc
4,HWCS 2020 Round 1 - Finn,Finn,180a9561-4034-4cc6-aaa6-698338bfd5cc
5,HWCS 2020 Round 1 - Laser,Laser Int.,180a9561-4034-4cc6-aaa6-698338bfd5cc
6,HWCS 2020 Round 1 - Laser Radial,Laser Radial,180a9561-4034-4cc6-aaa6-698338bfd5cc
7,HWCS 2020 Round 1 - Nacra 17,Nacra 17,180a9561-4034-4cc6-aaa6-698338bfd5cc
8,HWCS 2020 Round 1 - RS:X Men,RS:X,180a9561-4034-4cc6-aaa6-698338bfd5cc
9,HWCS 2020 Round 1 - RS:X Women,RS:X,180a9561-4034-4cc6-aaa6-698338bfd5cc


## races

In [18]:
sql = """
SELECT top (0) race_id, regatta, race from races
"""
raceSelection = pd.read_sql(sql, con = engine)
for idx, row in reg.iterrows():
    sql = """
    SELECT race_id, regatta, race from races
    where regatta = '{}'
    """.format(row.regatta)
    raceSelection = raceSelection.append(pd.read_sql(sql, con = engine))
raceSelection = raceSelection.reset_index(drop = True)
print(raceSelection)
stop

                                               race_id  \
0    b'\x11Q\x13\x00\xad\xb9\x017\x17\x15\x06w?\x91rv'   
1    b'\x13\x80\x06\xc0\xad\xb9\x017\x17^\x06w?\x91rv'   
2          b'y\x13\x9c@\xa9\xf8\x017\xb9~\x06w?\x91rv'   
3    b'\x8ae\xe8\x00\xac\xf1\x017\x04\xdf\x06w?\x91rv'   
4       b'\x8ca\x1a\x90\xac\xf1\x017\x05(\x06w?\x91rv'   
..                                                 ...   
285        b'\x8fJ\xf6\x80\xa4\x9e\x017\xac-H\xf7PH#*'   
286        b'\x8f\xca;\xb0\xa4\x9e\x017\xacCH\xf7PH#*'   
287        b'\x90T\xa1\xe0\xa4\x9e\x017\xacYH\xf7PH#*'   
288        b'\xd3\xd0\x02\x90\xa5n\x017\xbcnH\xf7PH#*'   
289           b'\xe5E\x1a\xa0\xa2\xc5\x017^oH\xf7PH#*'   

                         regatta                    race  
0    HWCS 2020 Round 1 - 470 Men           R09 (470 Men)  
1    HWCS 2020 Round 1 - 470 Men           R10 (470 Men)  
2    HWCS 2020 Round 1 - 470 Men            R1 (470 Men)  
3    HWCS 2020 Round 1 - 470 Men            R6 (470 Men)  
4    HWC

NameError: name 'stop' is not defined

# Fill columns with extra information to complete database


## Update final rank in race_comp


In [ ]:

for indx, row in raceSelection.iterrows():    
    regName = row.regatta
    raceName = row.race    
    race_id = idToText(row.race_id)
    print(regName, raceName)

    sql = """
    UPDATE
        race_comp
    SET
        race_comp.rank = (
                SELECT comp_leg.competitor_rank from comp_leg 
                        INNER JOIN legs ON comp_leg.race_id = legs.race_id
                                    AND comp_leg.leg_nr = legs.leg_nr
                WHERE legs.leg_nr_from_finish = 1
                    AND race_comp.race_id = comp_leg.race_id 
                    AND race_comp.comp_id = comp_leg.comp_id

    )
    where race_comp.race_id = {race_id}
    """.format(race_id = race_id)
    cursor.execute(sql)
    cnxn.commit()


## Get chosen side of competitors for leg 1

In [ ]:
for indx, row in raceSelection.iterrows():
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    
    print('Get competitors in leg 1', end = '\r')
    sql = """SELECT  comp_leg.race_id, comp_leg.comp_id
                    ,comp_leg.competitor_started
                    ,comp_leg.competitor_finished
                    ,comp_leg.avg_side

            from comp_leg 
            where comp_leg.race_id = {}
                  and leg_nr = 1""".format(race_id)
    df = pd.read_sql_query(sql, con = engine)
    for index, row in df.iterrows():
        
        
        comp_id = "0x{}".format(row['comp_id'].hex().upper())
        sql = """SELECT  TOP (1) *
          FROM comp_leg
          WHERE race_id = {}
          and comp_id = {}
          and leg_nr = 1""".format(race_id, comp_id)
        comp_leg = pd.read_sql_query(sql, con = engine, chunksize=None)

        if not (comp_leg.avg_side.values[0] is None):
#             print(row['regatta'], row['race'] , comp_id , end="\t")
            print('{} {} {} Side already determined'.format(regName, raceName, comp_id))
            continue

        if not row['competitor_started'] or not row['competitor_finished']:
#             print(row['regatta'], row['race'] , comp_id , end="\t")
            print('{} {} {} Competitor did not finish leg'.format(regName, raceName, comp_id))
            continue
        
        print('Get track of competitor', end = '\r')
        sql = """SELECT  *
          FROM positions
          WHERE leg_nr = 1
          and race_id = {}
          and comp_id = {}
          """.format(race_id, comp_id)
        track = pd.read_sql_query(sql, con = engine, chunksize=None)


        if track.empty:
            print('No track found {} {} {}'.format(regName, raceName, comp_id))
            break
            
            
        print('Determine stats\t\t\t', end = '\r')
        stats = [str(x) for x in trackStatistics(track)]
        print('Update \t\t\t\t\t', end = '\r')
        cursor.execute("""
        UPDATE comp_leg
        SET    comp_leg.avg_side = {}
               ,comp_leg.most_left = {}
               ,comp_leg.most_right = {}
        WHERE
        race_id = {}
          and comp_id = {}
          and leg_nr = 1
          """.format(stats[0], stats[1], stats[2],race_id, comp_id)
                      )
        cursor.commit()
        
    print('Succesfully updated {} {}'.format(regName, raceName))



## Determining Course Area

In [ ]:
sql = """SELECT  *
  FROM course_areas"""
areas = pd.read_sql_query(sql, con = engine)

areas['equid'] = areas.apply(lambda x:  coordinatesToEquiv(x, centre = False), axis =  1)


for indx, row in raceSelection.iterrows():
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    

    print('determining racearea of {} {}'.format(regName,raceName)
          , end = '\r')
    value = str(getArea(row, areas, engine))
    if value == "-1":
        continue
    print(value)
    print('update', end = '\r')
    cursor.execute("""UPDATE races
                    SET races.course_area_id = {}
                    WHERE race_id = {}
                    """.format(value, race_id))
    cursor.commit()




## Number of competitors per race and their relative rank

In [ ]:
for indx, row in raceSelection.iterrows():
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    print(regName, raceName)
    cursor.execute("""
                    UPDATE
                        races
                    SET
                        races.nr_competitors = temp_table.nr_competitors
                    FROM
                        (SELECT race_comp.race_id, count(rank) as nr_competitors
                                                  FROM race_comp 
                                                  GROUP by race_id) AS temp_table
                        INNER JOIN races on temp_table.race_id = races.race_id
                    WHERE races.race_id = {}
                """.format(race_id))
    cursor.commit()

In [ ]:
for indx, row in raceSelection.iterrows():
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    print(regName, raceName)
    cursor.execute("""
                UPDATE
                    comp_leg
                SET
                    comp_leg.rel_rank = temp_table.rel_rank
                FROM
                    (SELECT comp_leg.race_id, comp_leg.comp_id
                           ,comp_leg.competitor_rank
                           , comp_leg.competitor_rank/convert(decimal(4,2), races.nr_competitors) as rel_rank
                        FROM comp_leg INNER JOIN races
                            ON comp_leg.race_id = races.race_id 
                        )
                        AS temp_table INNER JOIN comp_leg
                            ON comp_leg.race_id = temp_table.race_id 
                                 and comp_leg.comp_id = temp_table.comp_id
                where comp_leg.race_id = {}

                """.format(race_id))
    cursor.commit()

In [ ]:
race_id

## Categorize Sailing Style per Race

In [ ]:
# regName = 'HWCS 2020 Round 1 - 470 Men'
# sql = """SELECT  *
#   FROM legs
#   WHERE regatta = '{}'
#   and leg_nr = 1""".format(regName)
# df = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)
# race = df.loc[4,:]
# print(race)
# sql = """SELECT  *
#   FROM sailing_style"""
# styles = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)
# style = styles.loc[1,:]
# style

In [ ]:
# for serv in eventSelection:
#     for indx, regatta in serv['regattas'].iterrows():
#         print('determining sailing types of {}'.format(regName)
#         regName = regatta.regatta
#         sql = """SELECT  *
#           FROM legs
#           WHERE legs.regatta = '{}'
#           and leg_nr = 1""".format(regName)
#         df = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True, params=None, parse_dates=None, chunksize=None)
        
#         for ind, race in df.iterrows():
#                   , end = ' ')
#             print(race.correlation_tacks)
#             if isnan(race.correlation_tacks):
#                 continue
#             value = sailing_style(race, styles)
#             print(value)
#             cursor.execute("""UPDATE races
#                             SET races.sail_style_id = {}
#                             WHERE regatta = '{}'
#                                 AND race = '{}'
#                             """.format(value, regName, race['race']))
#             cursor.commit()
    


## Positions Start Line

In [ ]:
## get from new positions

In [ ]:
for indx, row in raceSelection.iterrows():
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    print(regName, raceName, end = '\r')
    ## Start time
    sql = """SELECT  *
      FROM races
      WHERE race_id = {}""".format(race_id)
    race = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True,
                             params=None, parse_dates=None, chunksize=None)

    st = race['startOfRace-ms'].values[0] ## start time
    if st is None:
        print('\t No start time found for {} {}'.format(regName, raceName))
        continue


    print('Updating Startline', end = '\r')


    ## position buoys
    sql = """SELECT  *
      FROM courses
      WHERE race_id = {}
      and mark_nr = 0""".format(race_id)
    start = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True,
                              params=None, parse_dates=None, chunksize=None)

    id_l = start['controlPoint.left.id'].values[0]  #RC
    id_r = start['controlPoint.right.id'].values[0] #Pin

    sql = """SELECT  *
      FROM marks_positions
      WHERE race_id = {}
      and (id = {} or id = {})
      and [timepoint-ms] <= {}+3000
      and [timepoint-ms] >= {}-3000""".format(race_id, idToText(id_l), idToText(id_r), st,st)
    st_line = pd.read_sql_query(sql, con = engine)
    #select one timepoint
    if st_line.empty:
        print("\tNo startline available for {} {}".format(regName, raceName))
        continue
    mask = st_line[['id', 'timepoint-ms']].groupby('id').idxmin()['timepoint-ms']
    st_line = st_line.loc[mask].set_index('id')            
    
    if len(st_line)<2:
        print('Startline not complete for {} {}'.format(regName, raceName))
        continue
    rc = st_line.loc[id_l,:]
    pin = st_line.loc[id_r,:]
    st_bearing = bearing(rc,pin)        
        
    ## windmeasurements at start
    skip_wind = 0
    sql = """SELECT * FROM windsources
                    WHERE race_id = {}
                        AND typeName = 'EXPEDITION'""".format(race_id)
    windsources = pd.read_sql_query(sql, con = engine)
    rc_windsource = windsources.loc[windsources.rc == 1]
    
    if rc_windsource.empty:
         ## wind measurement at RC
        print('Get windsources', end = '\r')
        sql = """SELECT  *
            FROM wind
            WHERE race_id = {}
            and [timepoint-ms] <= {}+3000
            and [timepoint-ms] >= {}-3000
            """.format(race_id, st, st)
        wind = pd.read_sql_query(sql, con = engine)
        wind = wind.loc[wind.windSource == 'EXPEDITION']
        if wind.empty:
            print('No wind measurements available for {} {}'.format(regName, raceName))
            skip_wind = 1
        
        else:
            print('Determine closest ws', end = '\r')
            rc = st_line.loc[id_l,:]
            dist = wind[['lat-deg', 'lng-deg']].apply(lambda x: np.linalg.norm([x['lat-deg']- rc['lat-deg'],
                                                                                x['lng-deg']- rc['lng-deg']]),
                                               axis = 1)

            source = wind.loc[dist.idxmin()] 
            ## Update windsources to highlight this source to be at the rc
            sql = """UPDATE windsources
                            SET windsources.rc = 1
                            WHERE race_id = {}
                                AND typeName = 'EXPEDITION'
                                AND id = '{}'
                            """.format(race_id, source.windSource_id )
            cursor.execute(sql)
            cursor.commit()

    else:
        print('Wind at rc is already determined', end = '\r')
        # So the closest wind source is already known
        sql = """SELECT  *
            FROM wind
            WHERE race_id = {}
            and windSource = 'EXPEDITION'
            and windSource_id = '{}'
            """.format(race_id, rc_windsource.id.values[0])
        wind = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True, params=None,
                                 parse_dates=None, chunksize=None)
        source = wind.loc[0]
    
    if not skip_wind:
        wind_msr = wind.loc[(wind.windSource_id == source.windSource_id) & (wind['timepoint-ms'] == st)].reset_index()
                    ### check if there always exists one
        if wind_msr.empty:
            wind_msr = wind.loc[(wind.windSource_id == source.windSource_id)].reset_index()
            wind_time = wind_msr['timepoint-ms'].apply(lambda x: abs(x - st)).idxmin()
            wind_msr = wind_msr.loc[wind_time]

        ## determine startline stats
        wind_dir = wind_msr.squeeze()['dampenedTrueBearing-deg']
        dir_diff = wind_dir - (st_bearing - 90) 
        dir_diff = ((dir_diff+ 180)%360) - 180 ## denk na over modulo rekenen, zodat alles tussen -40 + 40 komt

        if dir_diff <-90:
            dir_diff +=180
        if dir_diff >90:
            dir_diff -= 180
        if dir_diff<0:
            side = 'Left'
        else:
            side = 'Right'
#             plt.plot(rc['lng-deg'], rc['lat-deg'], 'r.')
#             plt.plot(pin['lng-deg'], pin['lat-deg'], 'b.')
#             plt.show()
#             print('st_bearing = {}\t perpendicular to st = {}\t windbearing = {}\t calculated angle = {}'.format(st_bearing,
#                                                                                                                 (st_bearing - 90), 
#                                                                                                                 wind_msr.squeeze()['trueBearing-deg'],
#                                                                                                                 dir_diff))
        ## update races table
        sql = """UPDATE races
                        SET races.stl_bearing_diff_wind = {}
                        , races.stl_fav_side = '{}'
                        , races.start_wind_dir = {}
                        , races.stl_bearing = {}
                        WHERE race_id = {}
                        """.format(dir_diff, side, wind_dir, st_bearing,race_id)
        cursor.execute(sql)
        cursor.commit()

    print('Updating Competitors at Start    ', end = '\r')
    # check whether competitors are already updated
    sql =""" SELECT * from race_comp
                    WHERE race_id = {}
                        AND pos_startline_abs_x IS NOT NULL                               

                    """.format(race_id)
    df = pd.read_sql(sql, con = engine)
    if not df.empty:
        print('competitor start locations already known for {} {}'.format(regName, raceName))
        continue



    ## positions competitors at start
    sql = """select p.*
            from positions p
            join (select race_id, comp_id, min(timepoint_ms) as t
                  from positions
                  group by race_id, comp_id) p2
                on p.race_id = p2.race_id and p.comp_id = p2.comp_id 
                and p.timepoint_ms = p2.t
            Inner join comp_leg as cl on 
                    p.race_id = cl.race_id and p.comp_id = cl.comp_id 
                    and p.leg_nr = cl.leg_nr
            where 
            p.race_id = {}
            and cl.competitor_started = 1 and cl.leg_nr = 1
    """.format(race_id)
    comp = pd.read_sql_query(sql, con = engine, index_col=None, coerce_float=True, params=None,
                             parse_dates=None, chunksize=None)
    if comp.empty:
        print('WARNING: No positions found for competitors')

    X = startline(comp, pin, rc)
    X_x =  np.asarray(X)[0]
    X_y = np.asarray(X)[1]
#     plt.plot(X_x, X_y, '.')
#     plt.axis('equal')
#     plt.show()
    comp['pos_startline_abs_x'] = X_x
    comp['pos_startline_abs_y'] = X_y
    rel_order = X_x.argsort().argsort()/(len(X_x)-1)
    comp['pos_startline_rel'] = rel_order

    ## Update race_comp
    for idx, competitor in comp.iterrows():
        sql = """UPDATE race_comp
                    SET race_comp.pos_startline_abs_x = {}
                    , race_comp.pos_startline_abs_y = {}
                    , race_comp.pos_startline_rel = {}
                    WHERE race_id = {}
                        AND comp_id = {}
                    """.format(competitor.pos_startline_abs_x,
                               competitor.pos_startline_abs_y,
                               competitor.pos_startline_rel,
                               race_id,
                              idToText(competitor.comp_id))
        cursor.execute(sql)
        cursor.commit()

    print('Succesfully updated startline for {} {}'.format(regName,raceName))


## Relative Speeds


In [15]:
for indx, row in raceSelection.iterrows():  ## per race
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    print(regName, raceName, end = '\r')
    
    
    
    sql = """
    SELECT * from legs
    where race_id = {}
    and leg_nr = 1
    """.format(race_id)
    legs = pd.read_sql(sql, engine)
    
    for idx, leg in legs.iterrows():  ## per leg
        leg_nr = leg.leg_nr
        print(leg_nr, end = '\r')
        
        sql = """
        SELECT TOP(1) * from positions
        where leg_nr = {}
         and race_id = {}
        """.format(leg_nr, race_id)
        check = pd.read_sql(sql, engine)
        if check.empty:
            print('No positions available')
            continue
            
        if not check.iloc[0]['rel_spd-kts'] is None:
            print('already updated')
            continue

        sql = """
        SELECT race_id, comp_id, leg_nr, [competitor_averageSOG-kts] from comp_leg
        where race_id = {}
        and leg_nr = {}
        """.format( race_id, leg_nr)
        df = pd.read_sql(sql, engine)
        avg_spd = df['competitor_averageSOG-kts'].mean()
        
        if isnan(avg_spd):
            continue
            
        sql = """
        UPDATE legs
        SET avg_spd = {}
        WHERE race_id = {}
        and leg_nr = {}
        """.format(avg_spd, race_id, leg_nr)
        cursor.execute(sql)
        cursor.commit()
        
        print('update positions {}'.format(leg_nr), end = '\r')
        sql = """
        UPDATE positions
        set [rel_spd-kts] = [speed-kts]/{}
        where leg_nr = {}
        and race_id = {}
        """.format(avg_spd, leg_nr, race_id)
        cursor.execute(sql)
        cursor.commit()
        
    print('Succesfuly updated {} {}'.format(raceName, regName))
        

already updatedser Radial R4 (Laser Radial)
Succesfuly updated R4 (Laser Radial) Tokyo 2019 - Laser Radial
already updatedser Radial R5 (Laser Radial)
Succesfuly updated R5 (Laser Radial) Tokyo 2019 - Laser Radial
already updatedser Radial R6 (Laser Radial)
Succesfuly updated R6 (Laser Radial) Tokyo 2019 - Laser Radial
already updatedser Radial R2 (Laser Radial)
Succesfuly updated R2 (Laser Radial) Tokyo 2019 - Laser Radial
already updatedser Radial R3 (Laser Radial)
Succesfuly updated R3 (Laser Radial) Tokyo 2019 - Laser Radial
already updatedser Radial R7 (Laser Radial)
Succesfuly updated R7 (Laser Radial) Tokyo 2019 - Laser Radial
already updatedser Radial R8 (Laser Radial)
Succesfuly updated R8 (Laser Radial) Tokyo 2019 - Laser Radial
Succesfuly updated R9 (Laser Radial) Tokyo 2019 - Laser Radial
Succesfuly updated R10 (Laser Radial) Tokyo 2019 - Laser Radial
Succesfuly updated R1 (Laser Radial) Tokyo 2019 - Laser Radial
Succesfuly updated R6 (49er) Tokyo 2019 - 49er
Succesfuly upd

In [25]:
for indx, row in raceSelection.iterrows():  ## per race
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    print(regName, raceName, end = '\r')
    
    sql = """
    SELECT * from legs
    where race_id = {}
    and leg_nr = 1
    """.format(race_id)
    legs = pd.read_sql(sql, engine)
    
    for idx, leg in legs.iterrows():  ## per leg
        leg_nr = leg.leg_nr
        print(leg_nr, end = '\r')
        
        sql = """
        SELECT TOP(1) * from comp_leg
        where leg_nr = {}
         and race_id = {}
        """.format(leg_nr, race_id)
        check = pd.read_sql(sql, engine)
            
        if not check.iloc[0]['rel_averageSOG'] is None:
            print('already updated')
            continue

        sql = """
        SELECT race_id, comp_id, leg_nr,[competitor_averageSOG-kts] from comp_leg
        where race_id = {}
        and leg_nr = {}
        """.format( race_id, leg_nr)
        df = pd.read_sql(sql, engine)
        avg_dt = df['competitor_averageSOG-kts'].mean()
        
        if isnan(avg_dt):
            continue
            

        
        print('update positions {}'.format(leg_nr), end = '\r')
        sql = """
        UPDATE comp_leg
        set rel_averageSOG = [competitor_averageSOG-kts]/{}
        where leg_nr = {}
        and race_id = {}
        """.format(avg_dt, leg_nr, race_id)
        cursor.execute(sql)
        cursor.commit()
        
    print('Succesfuly updated {} {}'.format(raceName, regName))
        

already updated 1 - 470 Men R09 (470 Men)
Succesfuly updated R09 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R10 (470 Men)
Succesfuly updated R10 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R1 (470 Men)
Succesfuly updated R1 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R6 (470 Men)
Succesfuly updated R6 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R7 (470 Men)
Succesfuly updated R7 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R08 (470 Men)
Succesfuly updated R08 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R2 (470 Men)
Succesfuly updated R2 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men M Medal (470 Men)
Succesfuly updated M Medal (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R3 (470 Men)
Succesfuly updated R3 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R4 (470 Men)
Succesfuly updated R4 (470 Men) HW

Succesfuly updated R9 (RS:X Men) Tokyo 2019 - RS:X Men
Succesfuly updated R10 (RS:X Men) Tokyo 2019 - RS:X Men
Succesfuly updated R11 Medal (RS:X Men) Tokyo 2019 - RS:X Men
Succesfuly updated R1 (RS:X Men) Tokyo 2019 - RS:X Men
Succesfuly updated R6 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R7 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R8 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R2 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R3 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R4 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R5 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R9 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R10 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R11 (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R12 Medal (RS:X Women) Tokyo 2019 - RS:X Women
Succesfuly updated R1 (RS:X Women) Tokyo 2019 - RS:X Women


# Relative traveled distance

In [23]:
for indx, row in raceSelection.iterrows():  ## per race
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    print(regName, raceName, end = '\r')
    
    sql = """
    SELECT * from legs
    where race_id = {}
    and leg_nr = 1
    """.format(race_id)
    legs = pd.read_sql(sql, engine)
    
    for idx, leg in legs.iterrows():  ## per leg
        leg_nr = leg.leg_nr
        print(leg_nr, end = '\r')
        
        sql = """
        SELECT TOP(1) * from comp_leg
        where leg_nr = {}
         and race_id = {}
        """.format(leg_nr, race_id)
        check = pd.read_sql(sql, engine)
            
        if not check.iloc[0]['rel_distanceTraveled'] is None:
            print('already updated')
            continue

        sql = """
        SELECT race_id, comp_id, leg_nr,[competitor_distanceTraveled-m] from comp_leg
        where race_id = {}
        and leg_nr = {}
        """.format( race_id, leg_nr)
        df = pd.read_sql(sql, engine)
        avg_dt = df['competitor_distanceTraveled-m'].mean()
        
        if isnan(avg_dt):
            continue
            
        sql = """
        UPDATE legs
        SET [avg_distanceTraveled-m] = {}
        WHERE race_id = {}
        and leg_nr = {}
        """.format(avg_dt, race_id, leg_nr)
        cursor.execute(sql)
        cursor.commit()
        
        print('update comp_leg {}'.format(leg_nr), end = '\r')
        sql = """
        UPDATE comp_leg
        set rel_distanceTraveled = [competitor_distanceTraveled-m]/{}
        where leg_nr = {}
        and race_id = {}
        """.format(avg_dt, leg_nr, race_id)
        cursor.execute(sql)
        cursor.commit()
        
    print('Succesfuly updated {} {}'.format(raceName, regName))
        

already updated 1 - 470 Men R09 (470 Men)
Succesfuly updated R09 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R10 (470 Men)
Succesfuly updated R10 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R1 (470 Men)
Succesfuly updated R1 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R6 (470 Men)
Succesfuly updated R6 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R7 (470 Men)
Succesfuly updated R7 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R08 (470 Men)
Succesfuly updated R08 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R2 (470 Men)
Succesfuly updated R2 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men M Medal (470 Men)
Succesfuly updated M Medal (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R3 (470 Men)
Succesfuly updated R3 (470 Men) HWCS 2020 Round 1 - 470 Men
already updated 1 - 470 Men R4 (470 Men)
Succesfuly updated R4 (470 Men) HW

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\Nerine.USMAN2\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\Nerine.USMAN2\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\Nerine.USMAN2\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 530, in do_rollback
    dbapi_connection.rollback()
KeyboardInterrupt


KeyboardInterrupt: 

## positions_view

In [ ]:
CREATE View positions_view
WITH SCHEMABINDING
AS 
SELECT
    rc.race_id,
    l.leg_nr,
    p.comp_id,
    timepoint_ms,
    [lat-deg],
	[lng-deg]
	,[speed-kts]
	,[truebearing-deg]
    , [rel_spd-kts]
FROM
    dbo.positions p
--INNER JOIN dbo.race_comp c
--    ON c.comp_id = p.comp_id
--	 and c.race_id = p.race_id
INNER JOIN dbo.legs l
	ON l.leg_nr = p.leg_nr
	and l.race_id = p.race_id
INNER JOIN dbo.races rc 
    ON rc.race_id = l.race_id
WHERE l.leg_nr = 1
 and p.timepoint_ms % 20000 = 0
 and (rc.regatta LIKE  '%HWCS 2020 Round 1%' 
	or rc.regatta LIKE  '%Tokyo%'  
	or rc.regatta LIKE  '%Enoshima%'  
	 ;
        
CREATE UNIQUE CLUSTERED INDEX idx on dbo.positions_view (leg_nr, race_id, comp_id, timepoint_ms);

CREATE NONCLUSTERED INDEX 
    idx_regatta
ON dbo.positions_view(race_id);

## Calculate Correlations

In [ ]:
## not yet right for other legs then 1, because avg side needs to be calculated first

    
for indx, row in raceSelection.iterrows():
    regName = row.regatta
    raceName = row.race
    race_id = idToText(row.race_id)
    print(regName, raceName, end = '\r')

    stop = 0
    sql = """
    SELECT * from legs
    where leg_nr = 1
        AND race_id = {}
        """.format(race_id)

    legs = pd.read_sql(sql, engine)
    for index, row in legs.iterrows():
        
        sql = """
        SELECT comp_leg.*
                , race_comp.pos_startline_abs_x
                , race_comp.pos_startline_abs_y
                ,race_comp.pos_startline_rel 
                    from comp_leg INNER JOIN race_comp
        ON race_comp.race_id = comp_leg.race_id
            and race_comp.comp_id = comp_leg.comp_id          
        WHERE comp_leg.race_id = {}
        and comp_leg.leg_nr = {}
        """.format( race_id,
                    row['leg_nr'])

        df = pd.read_sql(sql, engine)
        correlations = df.corr()
        try:
            corrs = correlations.rel_rank.replace(np.nan, "NULL", regex=True)  ### replace NaN's by NULL so they can be updated in the sql df

            corrs_side = correlations.avg_side
        except AttributeError as a:
            print('Attribute Error: {} for {} {}'.format(a, regName, raceName))
        try:
            cursor.execute("""
            UPDATE legs
            SET    legs.correlation_tacks = {}
                  ,legs.correlation_side = {}
                  ,legs.correlation_avgSOG = {}
                  ,legs.correlation_traveledDistance = {}
                  ,legs.correlation_jibes = {}
                  ,legs.pos_startline_abs_x = {}
                  ,legs.pos_startline_abs_y = {}
                  ,legs.pos_startline_rel = {}
                  ,legs.cor_side_pos_startline_abs = {}
                  ,legs.cor_side_pos_startline_rel = {}
            WHERE
            race_id = {}
            and leg_nr = '{}'
              """.format(corrs['competitor_tacks'],
                         corrs['avg_side'],
                         corrs['competitor_averageSOG-kts'],
                         corrs['competitor_distanceTraveled-m'],
                         corrs['competitor_jibes'], 
                         corrs['pos_startline_abs_x'], 
                         corrs['pos_startline_abs_y'], 
                         corrs['pos_startline_rel'],  
                         corrs_side.pos_startline_abs_x,
                         corrs_side.pos_startline_rel,
                         race_id,
                         row['leg_nr']))        
        except KeyError as e:
            print('Key Error: {} for {} {}'.format(e, regName, raceName))
            stop = 1
        cursor.commit()
        print('{} {} Succesfull'.format(regName, raceName))

        


In [ ]:
sql = """
SELECT * from legs 
where pos_startline_abs_x IS NULL and
correlation_side is not null
and leg_nr = 1

"""
df = pd.read_sql(sql, con = engine)
df



# Extra

## Usefull other queries

In [ ]:
cursor.execute("""

USE nerinedatabase;
EXEC sp_spaceused;
""")


In [ ]:
cursor.execute("""
DROP INDEX idx ON positions;
CREATE UNIQUE CLUSTERED INDEX idx ON positions (regatta, race, comp_id, timepoint_ms ASC);

""")


In [ ]:
# info over indeces
EXEC sp_helpindex '[[[SCHEMA-NAME.TABLE-NAME]]]'
GO

## snelle manier om aantal rijen in tabel te krijgen
DECLARE @TableName sysname
SET @TableName = 'positions'
 
SELECT OBJECT_NAME(object_id), SUM(row_count) AS rows
FROM sys.dm_db_partition_stats
WHERE object_id = OBJECT_ID(@TableName)
AND index_id < 2
GROUP BY OBJECT_NAME(object_id);

## get all table and column names in db

In [36]:
sql = """SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
"""

df = pd.read_sql(sql, engine)

In [49]:
tables = df
# tables.to_latex()
print(tables[['TABLE_NAME', 'COLUMN_NAME', 'DATA_TYPE' ]].set_index(['TABLE_NAME', 'COLUMN_NAME']).to_latex())

\begin{tabular}{lll}
\toprule
            &            &  DATA\_TYPE \\
TABLE\_NAME & COLUMN\_NAME &            \\
\midrule
courses & race\_id &     binary \\
            & name &    varchar \\
            & passingInstruction &    varchar \\
            & controlPoint.@class &    varchar \\
            & controlPoint.name &    varchar \\
            & controlPoint.id &     binary \\
            & controlPoint.left.@class &    varchar \\
            & controlPoint.left.name &    varchar \\
            & controlPoint.left.id &     binary \\
            & controlPoint.left.type &    varchar \\
            & controlPoint.right.@class &    varchar \\
            & controlPoint.right.name &    varchar \\
            & controlPoint.right.id &     binary \\
            & controlPoint.right.type &    varchar \\
            & controlPoint.type &    varchar \\
            & mark\_nr &        int \\
            & mark\_nr\_from\_finish &        int \\
positions & race\_id &     binary \\
        